# Virtual Assistant

Connecting your chatbot to some source of external data source allows it to be more useful.
For example, linking to a rest API or database allows it to send queries.

The course discusses good practice in allowing users to amend query parameters. `sqllite3`
package is used and the course emphasises the safeguards that the `cursor.execute('SQL string ?', ?params)`
function has, preventing the users to send malicious queries that delete the database etc - SQL injection.

It is discouraged to use f strings or `.format()` for these same reasons.


In [1]:
import pandas as pd
from pyprojroot import here
import os

## Create a Database

In [2]:
from pathlib import Path
import sqlite3 # included in standard library

dbPath = os.path.join(here(), "data", "restaurants.db")
try:
    Path(dbPath).touch()
except:
    pass

In [3]:
con = sqlite3.connect(dbPath)
curs = con.cursor()

## Ingest restaurant Data

In [4]:
rests = pd.read_csv(
    "https://fsa-catalogue2.s3.eu-west-2.amazonaws.com/Approved+establishments+01-09-2022.csv",
    usecols=["TradingName", "Town", "Country", "CompetentAuthority"])


In [5]:
# create the sql table ready for write
curs.execute("""CREATE TABLE IF NOT EXISTS restaurants ('TradingName', 'Town', 'Country', 'CompetentAuthority');""")

## Write CSV to DataBase

In [6]:
# write the csv to the db
rests.to_sql('restaurants', con, if_exists='append', index = False)
del rests

In [7]:
params = ("Wales", "Food Standards Agency")
curs.execute("SELECT TradingName FROM restaurants WHERE Country = ? AND CompetentAuthority = ?", params)
curs.fetchall()

[('2 Sisters Food Group Limited T/A 2 Sisters Llangefini',),
 ('The Food Technology Centre',),
 ('Preseli Bluestone Meats Limited',),
 ('2 Sisters Food Group Limited T/A 2 Sisters Sandycroft',),
 ('Faccenda Foods Limited (T/A Avara Foods)',),
 ('Dunbia (UK)',),
 ('Llechwedd Meats',),
 ('Cig Eryri - Snowdonia Meats Cyf',),
 ("Daphne's Welsh Lamb Limited",),
 ('Celtica Foods Limited',),
 ('Bwydlyn',),
 ('Universal Foods',),
 ('M K (Wales) Ltd T/A (Maddock Kembery Meats)',),
 ('Weeks Wholesale Meat Company Ltd',),
 ('I & B Jones Ltd',),
 ('William Arthur James (T/A WA James Butchers)',),
 ('Maelor Foods Limited',),
 ('Hugh Phillips Gower Butcher',),
 ('Mid Glamorgan Provisions Ltd',),
 ('Farmers Pantry Ltd',),
 ('Williams Lloyd Williams and Son Limited',),
 ('WJ George (Butchers) Ltd',),
 ('Huw Evans & Morag Evans (T/A Cig Oen Caron)',),
 ('Direct Meat Supplies Ltd',),
 ('Conwy Valley Meats Limited',),
 ('David Thomas James Thomas Stephen Thomas & Simeon Thomas (T/A D & J Thomas)',),
 ('C

## Functional Approach

Define a function that can query the database based on parameters received.

In [75]:
def find_restaurants(param_dict, db_path):
    # Create the base query
    baseQuery = 'SELECT * FROM restaurants'
    # ensure params exists
    if len(param_dict) > 0:
        # Add filters for each params value
        param_filters = [f"{k}=?" for k in param_dict]
        filter_query = " WHERE " + " and ".join(param_filters)
    else:
        raise ValueError("params needs an argument.")
    
    # assemble the query
    query = baseQuery + filter_query

    # Create the tuple of values including the SELECT target first
    t = tuple(param_dict.values())
    # check db is valid
    if os.path.exists(db_path):
        # Open db connection
        conn = sqlite3.connect(db_path)
        # allow printing of queries for debugging
        conn.set_trace_callback("print")
        # Create cursor
        c = conn.cursor()
        # Execute query
        print(query)
        print(t)
        c.execute(query, t)
        return c.fetchall()
    else:
        raise FileNotFoundError(f"{db_path} not found.")
    return None
    


In [78]:
results = find_restaurants({"Country":"Wales", "CompetentAuthority":"Food Standards Agency"},
os.path.join(here(), "data", "restaurants.db"))

SELECT * FROM restaurants WHERE Country=? and CompetentAuthority=?
('Wales', 'Food Standards Agency')


### Respond

This section has a `rasa_nlu` dependency. An undisclosed interpreter is used to parse
messages and return parameter dictionaries to query. Investigating the interpreter
object yielded the following:
```
In [2]: interpreter
Out[2]: <rasa_nlu.model.Interpreter at 0x7fcd78e96f28>

In [3]: interpreter.context
Out[3]: {'spacy_nlp': <spacy.lang.en.English at 0x7fcd35e9b898>}

In [8]: interpreter.parse("Some message")
Out[8]:
{'entities': [{'end': 19,
   'entity': 'price',
   'extractor': 'ner_crf',
   'processors': ['ner_synonyms'],
   'start': 10,
   'value': 'hi'},
  {'end': 38,
   'entity': 'area',
   'extractor': 'ner_crf',
   'start': 33,
   'value': 'south'}],
 'intent': {'confidence': 0.0, 'name': ''},
 'text': 'some message'}
 ```

In [84]:
resps = ["Catchall: No matches found...",
"Only 1 result: {}",
"2 results found: {} or {}.",
"{} is one option, but there are more"]

In [86]:
rest_names = [r[0] for r in results]
# this bit compares the number of responses to the max number in your resp list, use it
# to select the appropriate response
index = min(len(results),3) 

In [87]:
# splat the results into the selected format string, return this result
resps[index].format(*rest_names)

'2 Sisters Food Group Limited T/A 2 Sisters Llangefini is one option, but there are more'

***

## Slot Filling

Slot filling is the term used to describe the chatbots memory to user input. If a user
needs to keep reiterating requirements in order to return relevant results, your bot has
the memory of a goldfish - this would be painful to have to interact with. Eg - 

> USER: Hi, is there a restaurant in Oz that serves vegan food?  
> BOT: I have lots of restaurants in Oz but none serve vegan food.  
> USER: How about a meal in Kansas instead?  
> <p style="color:red;">BOT: There's the Kansas karnivore steak house.</p> 

With a bit of incremental slot filling, your bot can use a dictionary o record entities 
or intents and respond with this instead:

> <p style="color:green;">BOT: How about Dot's Veggilicious CafeBar?</p>

